In [2]:
from langchain_groq import ChatGroq
import getpass
import os



In [3]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.kpmg.com.sg/job/ASPAC-Tax-Technology-and-Transformation-Manager/32443044/")
page_data = loader.load().pop().page_content
print(page_data)














ASPAC Tax Technology and Transformation Manager Job Details | KPMG Singapore Careers


























                    By continuing to use and navigate this website, you are agreeing to the use of cookies.
                
            

                Accept

                Close




Press Tab to Move to Skip to Content Link
Skip to main content
































Insights
Industries
Services
Careers
Events
About















Sign In / Create Account














































Insights
Industries
Services
Careers
Events
About















Sign In / Create Account












































Insights
Industries
Services
Careers
Events
About





















Sign In / Create Account



























Search by Keyword






Show More Options





Loading...







                                            Category
                                        


All





                              

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(type(res.content))
print(res.content)

<class 'str'>
```json
{
  "role": "ASPAC Tax Technology and Transformation Manager",
  "experience": "Experienced Hires",
  "skills": [
    "Robotics Process Automation (RPA)",
    "Data Science and Data Analytics (e.g. Python)",
    "Development using Microsoft stack",
    "Design and development of databases (specifically SQL Server)",
    "Developing ETL solutions using SSIS or other tools",
    "Data analytics and visualisation tools such as PowerBI",
    "Microsoft Power suite (Power App, Power Automate, Query, Macros and VBA, Scripting)"
  ],
  "description": "The role involves working on various client and internal tax technology projects, from assisting with internal production, automation and data analytics solutions, to implementing third-party tax applications for MNCs. The ideal candidate will have a bachelor's degree in a related discipline, excellent communication and business writing skills, strong organisational and time management skills, and a natural flair for proble

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'ASPAC Tax Technology and Transformation Manager',
 'experience': 'Experienced Hires',
 'skills': ['Robotics Process Automation (RPA)',
  'Data Science and Data Analytics (e.g. Python)',
  'Development using Microsoft stack',
  'Design and development of databases (specifically SQL Server)',
  'Developing ETL solutions using SSIS or other tools',
  'Data analytics and visualisation tools such as PowerBI',
  'Microsoft Power suite (Power App, Power Automate, Query, Macros and VBA, Scripting)'],
 'description': "The role involves working on various client and internal tax technology projects, from assisting with internal production, automation and data analytics solutions, to implementing third-party tax applications for MNCs. The ideal candidate will have a bachelor's degree in a related discipline, excellent communication and business writing skills, strong organisational and time management skills, and a natural flair for problem solving and an entrepreneurial approach to wor

In [10]:
type(json_res)

dict

In [ ]:
import pandas as pd

df = pd.read_csv("resources/my_portfolio.csv.csv")
df

,Technologies Used,Website Link
0,"Machine Learning, Supervised Learning, Scikit-...",https://github.com/JoelNgiamKeeYong/spacex-fal...
1,"Generative AI, NLP, LLM, LangChain, Streamlit,...",https://joelngiamkeeyong-crisis-management-age...
2,"Mobile Development, React Native, Expo, Fireba...",https://jim-app-fitech.web.app/
3,"Machine Learning, Reinforcement Learning, Deep...",https://joelngiamkeeyong-snake-game-ai-rl-app-...


In [13]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row['Technologies Used'],
            metadatas={'links': row['Website Link']},
            ids=[str(uuid.uuid4())]
        )

In [17]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://joelngiamkeeyong-snake-game-ai-rl-app-tn9faw.streamlit.app/'},
  {'links': 'https://joelngiamkeeyong-crisis-management-agent-llm-app-gkkssi.streamlit.app/'}],
 [{'links': 'https://jim-app-fitech.web.app/'},
  {'links': 'https://github.com/JoelNgiamKeeYong/spacex-falcon-9-mission-analytics'}]]

In [18]:
job = json_res
job['skills']

['Robotics Process Automation (RPA)',
 'Data Science and Data Analytics (e.g. Python)',
 'Development using Microsoft stack',
 'Design and development of databases (specifically SQL Server)',
 'Developing ETL solutions using SSIS or other tools',
 'Data analytics and visualisation tools such as PowerBI',
 'Microsoft Power suite (Power App, Power Automate, Query, Macros and VBA, Scripting)']

In [20]:
prompt_job_application = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    {job_description}

    ### INSTRUCTIONS
    - Write a professional and personalized cover letter tailored to the given job description.
    - Highlight relevant skills and experiences that align with the role.
    - Maintain a confident, enthusiastic, and engaging tone.
    - Ensure the letter is structured with an introduction, body, and conclusion.
    - Use a formal yet approachable style.
    - End with a strong closing statement expressing interest and willingness to discuss further.
    - Add portfolio links that matches the skills required for the job description from the portfolio: {link_list}

    Best regards,  
    Joel
    ```
    """
)

chain_email = prompt_job_application | llm
res = chain_email.invoke({
    "job_description": str(job),
    "link_list": links
})

print(res.content)

Dear Hiring Manager,

I am excited to apply for the ASPAC Tax Technology and Technology Manager position, as advertised. As a highly motivated and experienced professional with a passion for tax technology and innovation, I am confident that my skills and experience make me an ideal fit for this role.

With a strong background in Robotics Process Automation (RPA), Data Science and Data Analytics, and proficiency in programming languages such as Python, I am well-equipped to drive the development of internal production, automation, and data analytics solutions. My experience in designing and developing databases, specifically SQL Server, and developing ETL solutions using SSIS, has provided me with a solid foundation in data management. Additionally, I have expertise in data analytics and visualization tools such as PowerBI, and I am familiar with the Microsoft Power suite, including Power Apps, Power Automate, and Power Query.

As a creative problem solver with an entrepreneurial minds